In [4]:
from __future__ import print_function
from load_glove import *
import json
from collections import defaultdict
import numpy as np
import tensorflow as tf

In [23]:
GLOVE_LOC = '../../data/glove/glove.6B.50d.txt'
words = glove2dict(GLOVE_LOC)
word_counter = defaultdict(int)
GLV_DIM = words['the'].shape[0]
words = glove2dict(GLOVE_LOC)
word_counter = defaultdict(int)
GLV_DIM = words['the'].shape[0]

def clean(text,clip_n=0):
    res = text.replace('<d>','').replace('<p>','').replace('<s>','').replace('</d>','').replace('</p>','').replace('</s>','')
    
    r2 = []
    for word in res.split():
        if word not in words:
            words[word] = np.array([random.uniform(-0.5, 0.5) for i in range(GLV_DIM)])
    for word in res.split():
        word_counter[word] += 1
    if clip_n > 0:
        return ' '.join(res.split()[:clip_n])
    else:
        return res

In [24]:
SENTIMENT_LOC = '../../data/sentiment-treebank/data.json'

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Embedding, Dense, LSTM, Dropout,GRU
from keras.datasets import imdb

max_features = 20000
max_length = 80
embedding_dim = 256
batch_size = 128
epochs = 10

print('Loading data...')
def load_our_data():
    with open(SENTIMENT_LOC) as fp:
        data = json.load(fp)
        train = [x for x in data if x['set'] == 'train']
        dev = [x for x in data if x['set'] == 'dev']
        #test = [x for x in data if x['set'] == 'test']

        train = [(clean(x['data'],max_length),x['label'][0]) for x in train]
        dev = [(clean(x['data'],max_length),x['label'][0]) for x in dev]
        #test = [(clean(x['data'],INPUT_MAX),clean(x['label'][0],OUTPUT_MAX)) for x in test]

        valid_words = (sorted([(v,k) for k,v in word_counter.items()])[::-1])
        print(len(valid_words))
        valid_words = ['<PAD>'] + [x[1] for x in valid_words[:max_features-4]] + ['<EOS>','<UNK>','<SOS>']
        unk_idx = valid_words.index('<UNK>')
        vwd = defaultdict(lambda : unk_idx)
        for idx,word in enumerate(valid_words):
            vwd[word] = idx

        initial_matrix = np.array([words[x] for x in valid_words])
        def sent_to_idxs_nopad(sentence):
            base =  [vwd[word] for word in sentence.split()]
            base = base +  [0]*(max_length-len(base))
            return base
        train_x = np.array([sent_to_idxs_nopad(x[0]) for x in train if int(x[1]) != 0])
        train_y = np.array([int(x[1]>3) for x in train if int(x[1]) != 0])

        dev_x = np.array([sent_to_idxs_nopad(x[0]) for x in dev if int(x[1]) != 0])
        dev_y = np.array([int(x[1]>3) for x in dev if int(x[1]) != 0])

        #test_x = [sent_to_idxs_nopad(x[0]) for x in test]
        #test_y = [sent_to_idxs(x[1])[0] for x in test]
        #test_len = [sent_to_idxs(x[1])[1] for x in test]
        return (train_x,train_y),(dev_x,dev_y)

#(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)
(X_train, y_train), (X_test, y_test) = load_our_data()

X_train = sequence.pad_sequences(X_train, max_length)
X_test = sequence.pad_sequences(X_test, max_length)

# Compile and train different models while meauring performance.
results = []


model = Sequential()
model.add(Embedding(max_features, embedding_dim,
                    input_length=max_length))
#model.add(Dropout(0.2))
model.add(GRU(embedding_dim,
               #dropout=0.2,
               #recurrent_dropout=0.2,
               implementation=0))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

start_time = time.time()
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(X_test, y_test))
average_time_per_epoch = (time.time() - start_time) / epochs

results.append((history, average_time_per_epoch))



Loading data...
17601
Train on 7452 samples, validate on 962 samples
Epoch 1/10
7452/7452 [==============================] - 88s - loss: 0.4753 - acc: 0.8145 - val_loss: 0.4588 - val_acc: 0.8285

In [33]:
X_test.shape

(1101, 80)